In [1]:
import nbimporter
import gas_water
import moisture_conversion as mc
import property_conversion  as prop

Importing Jupyter notebook from gas_water.ipynb
Importing Jupyter notebook from moisture_conversion.ipynb
Importing Jupyter notebook from vapour_pressure.ipynb
Importing Jupyter notebook from property_conversion.ipynb
Importing Jupyter notebook from liquid_water.ipynb


# 液水の状態を保持するモジュール

液水の状態を保持するクラスを作成する。  
液水の状態を表す方法として以下の3つの方法が挙げられる。  

- $temp$ 絶対温度, K
- $\phi$ 含水率, %  
- $\mu$ 水分化学ポテンシャル, J kg<sup>-1<sup>  
- $P_l$ 溶液の圧力, Pa  

多孔質材料中では吸着力によりある量の液水が保持される。この時、液水の保持量（含水率）と液水の状態量（水分化学ポテンシャル）は材料に応じた相関がある。（平衡含水率関係）  
しかしこの平衡関係は材料ごとに異なるため、平衡関係を用いる際に材料の情報を入力する必要がある。ここでは水分化学ポテンシャル（液相）から含水率を決定するクラスと、含水率から水分化学ポテンシャル（液相）を決定するクラスの2つについて示す。

## クラス定義

### 1) 含水率ベース（気液平衡状態）

In [2]:
class PhiBasedWater():
    
    def __init__(self, temp, phi):
        self._temp = temp
        self._phi  = phi
        
    def set_material(self, material_name):
        self.material_name = material_name
    
    # temp
    
    def set_temp(self, temp):
        self._temp = temp
    
    def get_temp(self):
        return self._temp
    
    temp = property(get_temp, set_temp)

    # phi
    
    def set_phi(self, phi):
        self._phi = phi
        
    def get_phi(self):
        return self._phi
    
    phi = property(get_phi, set_phi)
    
    # miu
    
    def set_miu(self, miu):
        self._phi = prop.Kernel_Material_WaterAdsorption( self.material_name ).getPhi( MiuBasedWater(self.temp, miu) )
    
    def get_miu(self):
        return prop.Kernel_Material_WaterAdsorption( self.material_name ).getMiuByPhi( self )
    
    miu = property(get_miu, set_miu)
    

### 2) 水分化学ポテンシャルベース（気液平衡状態）

In [3]:
class MiuBasedWater():
    
    def __init__(self, temp, miu):
        self._temp = temp
        self._miu  = miu
    
    def set_material(self, material_name):
        self.material_name = material_name
        
    # temp
    
    def set_temp(self, temp):
        self._temp = temp
    
    def get_temp(self):
        return self._temp
    
    temp = property(get_temp, set_temp)

    # miu
    
    def set_miu(self, miu):
        self._miu = miu
        
    def get_miu(self):
        return self._miu
    
    miu = property(get_miu, set_miu)
    
    # phi
    
    def set_phi(self, phi):
        self._miu = prop.Kernel_Material_WaterAdsorption( self.material_name ).getMiuByPhi( PhiBasedWater(self.temp, phi) )
    
    def get_phi(self):
        return prop.Kernel_Material_WaterAdsorption( self.material_name ).getPhi( self )
    
    phi = property(get_phi, set_phi)
    

## 使い方

変数の宣言  
例として、
水蒸気の状態量を

- 絶対温度 300 K
- 相対湿度 0.9999(99.99%)  

とする。

In [4]:
vapour =  gas_water.RHBasedMoisture(temp=300.0, rh=0.9999)

材料の名前を'BentheimerSandstone'として気液平衡が成り立つとすると、  
液水のクラスは以下のように定義される。

In [5]:
water1 = MiuBasedWater( vapour.temp, vapour.miu)

In [6]:
water1.set_material('BentheimerSandstone')

水分化学ポテンシャルの取得

In [7]:
water1.miu

-13.848383372093412

含水率の取得

In [8]:
water1.phi

Importing Jupyter notebook from bentheimer_sandstone.ipynb
Importing Jupyter notebook from van_genuchten.ipynb


0.13286027466257927

含水率を与えて置き換え

In [9]:
water1.set_phi( 0.10 )

In [10]:
water1.miu

-20.29806887366382

同様に、含水率基準で考えると

In [11]:
water2 = PhiBasedWater( vapour.temp, 0.1)

In [12]:
water2.set_material('BentheimerSandstone')

含水率の取得

In [13]:
water2.phi

0.1

水分化学ポテンシャルを取得

In [14]:
water2.miu

-20.29806887366382

任意の水分化学ポテンシャルを与えて置き換え

In [15]:
water2.set_miu(-13.848383372093412)

In [16]:
water2.phi

0.13286027466257927